In [120]:
import db_connect
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from datetime import datetime
from scipy import stats

In [121]:
# read data
# trial with one city
earth_weather_data = db_connect.weather_data_location('Sydney')
mars_weather_data = db_connect.mars_weather_data()
solar_flares_data = db_connect.solar_flares_data()

Successfully connected to the database!
Successfully connected to the database!
Successfully connected to the database!


In [122]:
earth_weather_data.head()

,id,date,latitude,longitude,temperature_2m,relative_humidity_2m,rain,direct_radiation_instant,location
0,14763302,1940-01-04 16:00:00,37.9838,23.7275,7.82,68.928505,0.0,0.0,Sydney
1,14763303,1940-01-04 17:00:00,37.9838,23.7275,7.77,71.406982,0.0,0.0,Sydney
2,14763304,1940-01-04 18:00:00,37.9838,23.7275,7.87,75.837799,0.0,0.0,Sydney
3,14763305,1940-01-04 19:00:00,37.9838,23.7275,7.22,82.681374,0.2,0.0,Sydney
4,14763306,1940-01-04 20:00:00,37.9838,23.7275,5.52,86.648010,0.2,0.0,Sydney


In [123]:
earth_weather_data.date.min(), earth_weather_data.date.max()

(Timestamp('1940-01-01 07:00:00'), Timestamp('2024-03-16 23:00:00'))

In [124]:
earth_weather_data['date'] = pd.to_datetime(earth_weather_data['date'])

earth_weather_data['date'] = earth_weather_data['date'].dt.normalize()

# Group by the date and calculate the mean for each column of interest
daily_avg = earth_weather_data.groupby('date').agg({
    'latitude': 'first',
    'longitude': 'first',
    'temperature_2m': 'mean',
    'relative_humidity_2m': 'mean',
    'rain': 'mean',
    'direct_radiation_instant': 'mean',
    'location': 'first'
}).reset_index()

daily_avg.head()

,date,latitude,longitude,temperature_2m,relative_humidity_2m,rain,direct_radiation_instant,location
0,1940-01-01,37.9838,23.7275,2.120390,56.584553,0.054412,121.184209,Sydney
1,1940-01-02,37.9838,23.7275,2.150812,57.237792,0.038542,146.945197,Sydney
2,1940-01-03,37.9838,23.7275,1.431021,57.215635,0.005208,156.886286,Sydney
3,1940-01-04,37.9838,23.7275,2.614875,53.512488,0.032292,152.262421,Sydney
4,1940-01-05,37.9838,23.7275,3.349771,67.354769,0.250000,83.920918,Sydney


In [125]:
mars_weather_data.head()

,primary_key,id,terrestrial_date,sol,ls,month,min_temp,max_temp,pressure,atmo_opacity
0,19589,1895,2018-02-27,1977,135,Month 5,-77.0,-10.0,727.0,Sunny
1,19590,1893,2018-02-26,1976,135,Month 5,-77.0,-10.0,728.0,Sunny
2,19591,1894,2018-02-25,1975,134,Month 5,-76.0,-16.0,729.0,Sunny
3,19592,1892,2018-02-24,1974,134,Month 5,-77.0,-13.0,729.0,Sunny
4,19593,1889,2018-02-23,1973,133,Month 5,-78.0,-18.0,730.0,Sunny


In [126]:
mars_weather_data.terrestrial_date.min(), mars_weather_data.terrestrial_date.max()

(Timestamp('2012-08-16 00:00:00'), Timestamp('2022-11-25 00:00:00'))

In [127]:
solar_flares_data.head()

,id,flrid,instruments,begintime,peaktime,endtime,classtype,sourcelocation,activeregionnum,note,linkedevents,submissiontime,link,intensity
0,1144,2023-10-01T01:10:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2023-10-01 01:10:00,2023-10-01 01:32:00,2023-10-01 01:43:00,M2.5,N12E58,13452.0,None,None,2023-10-01T14:10Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,Medium
1,1145,2023-10-01T01:44:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2023-10-01 01:44:00,2023-10-01 01:46:00,2023-10-01 01:49:00,M1.6,N15E59,13451.0,None,None,2023-10-01T01:59Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,Medium
2,1146,2023-10-02T12:35:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2023-10-02 12:35:00,2023-10-02 12:46:00,2023-10-02 12:58:00,M1.9,N19E68,13455.0,None,None,2023-10-04T15:53Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,Medium
3,1147,2023-10-02T15:53:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2023-10-02 15:53:00,2023-10-02 15:58:00,2023-10-02 16:02:00,C2.4,S17E02,13450.0,None,[{'activityID': '2023-10-02T18:24:00-CME-001'}],2023-10-03T17:19Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,Small
4,1148,2023-10-07T17:57:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2023-10-07 17:57:00,2023-10-07 18:06:00,2023-10-07 18:10:00,M1.7,S12E28,13457.0,None,None,2023-10-07T18:24Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,Medium


In [128]:
# Select Relevant Features:
mars_weather_data = mars_weather_data[["terrestrial_date", "min_temp", "max_temp", "pressure"]]
earth_weather_data_daily = daily_avg[["date", "temperature_2m", "relative_humidity_2m", "rain", "direct_radiation_instant"]]
solar_flares_data = solar_flares_data[["peaktime", "classtype", "intensity"]]

# Rename Columns for Consistency:
mars_weather_data.rename(columns={"terrestrial_date": "date"}, inplace=True)
solar_flares_data.rename(columns={"peaktime": "date"}, inplace=True)

In [129]:
mars_weather_data['date'] = pd.to_datetime(mars_weather_data['date']).dt.date
earth_weather_data_daily['date'] = pd.to_datetime(earth_weather_data_daily['date']).dt.date
solar_flares_data['date'] = pd.to_datetime(solar_flares_data['date'], errors = 'coerce').dt.date

C:\Users\ansam\AppData\Local\Temp\ipykernel_32016\3253603673.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [130]:
solar_flares_data.columns

Index(['date', 'classtype', 'intensity'], dtype='object')

In [131]:
# Merge DataFrames:
merged_df = pd.merge(mars_weather_data, earth_weather_data_daily, on="date", how="left")
merged_df = pd.merge(merged_df, solar_flares_data, on="date", how="left")

In [132]:
merged_df.classtype.value_counts()

classtype
M1.0    108
M1.1     88
M1.2     73
M1.3     63
M1.4     53
       ... 
C2.0      1
B8.5      1
B7.5      1
B7.2      1
B6.1      1
Name: count, Length: 199, dtype: int64

In [133]:
merged_df['date'] = pd.to_datetime(merged_df['date']).dt.date
merged_df = merged_df.sort_values('date', ascending=False)
merged_df['solar_flare'] = merged_df['classtype'].notna().astype(int)
merged_df.head()


,date,min_temp,max_temp,pressure,temperature_2m,relative_humidity_2m,rain,direct_radiation_instant,classtype,intensity,solar_flare
2079,2022-11-25,-72.0,-44.0,817.0,4.435167,63.489129,0.018750,178.806240,NaN,NaN,0
2080,2022-11-24,-72.0,-9.0,800.0,4.676833,56.568758,0.000000,181.666258,NaN,NaN,0
2081,2022-11-23,-73.0,-8.0,797.0,5.531000,57.362091,0.080208,154.247278,NaN,NaN,0
2082,2022-11-22,-72.0,-10.0,798.0,4.671104,55.316284,0.038542,173.778638,C7.0,Small,1
2083,2022-11-22,-72.0,-10.0,798.0,4.671104,55.316284,0.038542,173.778638,C4.8,Small,1


In [134]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5450 entries, 2079 to 2078
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      5450 non-null   object 
 1   min_temp                  5449 non-null   float64
 2   max_temp                  5449 non-null   float64
 3   pressure                  5450 non-null   float64
 4   temperature_2m            5450 non-null   float64
 5   relative_humidity_2m      5450 non-null   float64
 6   rain                      5450 non-null   float64
 7   direct_radiation_instant  5450 non-null   float64
 8   classtype                 1259 non-null   object 
 9   intensity                 1259 non-null   object 
 10  solar_flare               5450 non-null   int32  
dtypes: float64(7), int32(1), object(3)
memory usage: 489.6+ KB


In [135]:
merged_df['intensity'] = merged_df['intensity'].fillna('None')
merged_df['classtype'] = merged_df['classtype'].fillna('None')

In [136]:
merged_df.tail(20)

,date,min_temp,max_temp,pressure,temperature_2m,relative_humidity_2m,rain,direct_radiation_instant,classtype,intensity,solar_flare
2059,2012-09-08,-73.0,-2.0,748.0,11.149333,59.942512,0.003125,195.980895,None,None,0
2060,2012-09-06,-74.0,-23.0,745.0,12.180583,58.509483,0.011458,196.106679,None,None,0
2061,2012-09-05,-74.0,-3.0,747.0,12.273292,59.553240,0.009375,160.138762,None,None,0
2062,2012-09-04,-75.0,-2.0,747.0,11.074854,62.005170,0.012500,179.320564,None,None,0
2063,2012-09-03,-75.0,-15.0,745.0,9.774854,67.953701,0.009375,181.360778,None,None,0
2064,2012-09-02,-75.0,-15.0,743.0,9.438396,70.284788,0.012500,148.950676,None,None,0
2065,2012-09-01,-76.0,-14.0,745.0,10.658708,62.514320,0.042708,158.041151,None,None,0
2066,2012-08-31,-75.0,-11.0,743.0,10.587875,57.166719,0.046875,179.563463,C8.4,Small,1
2067,2012-08-30,-75.0,-7.0,742.0,11.227458,56.489338,0.023958,171.775027,None,None,0
2068,2012-08-29,-75.0,-16.0,741.0,11.849333,61.995448,0.039583,156.630032,None,None,0


In [137]:
# Scale Data for Forecasting
scaler = MinMaxScaler()
merged_df[["min_temp", "max_temp", "pressure", "temperature_2m"]] = scaler.fit_transform(merged_df[["min_temp", "max_temp", "pressure", "temperature_2m"]])

In [138]:
# Time Series Plots:
merged_df['temperature_2m_smooth'] = merged_df['temperature_2m'].rolling(window=7).mean()

fig = make_subplots(rows=4, cols=1, shared_xaxes=True,
                    subplot_titles=("Mars Min Temp", "Mars Max Temp", "Mars Pressure", "Earth Temperature"))

# Plot each variable on a separate subplot
fig.add_trace(go.Scatter(x=merged_df['date'], y=merged_df['min_temp'], name='Mars Min Temp'), row=1, col=1)
fig.add_trace(go.Scatter(x=merged_df['date'], y=merged_df['max_temp'], name='Mars Max Temp'), row=2, col=1)
fig.add_trace(go.Scatter(x=merged_df['date'], y=merged_df['pressure'], name='Mars Pressure'), row=3, col=1)
fig.add_trace(go.Scatter(x=merged_df['date'], y=merged_df['temperature_2m_smooth'], name='Earth Temperature Smoothed'), row=4, col=1)

# Update x-axis and y-axis labels
fig.update_xaxes(title_text="Date", row=4, col=1)  # Only the last subplot needs the x-axis label
fig.update_yaxes(title_text="Degrees C", row=1, col=1)
fig.update_yaxes(title_text="Degrees C", row=2, col=1)
fig.update_yaxes(title_text="Pressure (Pa)", row=3, col=1)
fig.update_yaxes(title_text="Degrees C", row=4, col=1)

fig.update_layout(paper_bgcolor="#000000",
                  plot_bgcolor="#000000",
                  height=1000, 
                  width=1500, 
                  title_text="Time Series of Mars and Earth Weather Data")
fig.show()

In [141]:
fig = px.scatter(merged_df, x="min_temp", y="temperature_2m_smooth", color="solar_flare",
                  title="Mars Minimum Temperature vs. Earth Temperature (Solar Flare Impact)",
                  labels={"min_temp": "Mars Min Temp", "temperature_2m": "Earth Temperature"})
fig.update_layout(paper_bgcolor="#000000",
                  plot_bgcolor="#000000",
                  height=1000, 
                  width=2000)
fig.show()

In [142]:
# Scatter Plots:
fig = px.scatter(merged_df, x="min_temp", y="temperature_2m_smooth", color="classtype",
                  title="Mars Minimum Temperature vs. Earth Temperature (Solar Flare Impact by Class Type)",
                  labels={"min_temp": "Mars Min Temp", "temperature_2m": "Earth Temperature"},
                  category_orders={"classtype": sorted(merged_df['classtype'].unique())}) 
fig.update_layout(paper_bgcolor="#000000",
                  plot_bgcolor="#000000",
                  height=1000, 
                  width=2000)
fig.show()

In [143]:
fig = px.scatter(merged_df, x="min_temp", y="temperature_2m_smooth", color="intensity",
                  title="Mars Minimum Temperature vs. Earth Temperature (Solar Flare Impact by Intensity)",
                  labels={"min_temp": "Mars Min Temp", "temperature_2m_smooth": "Earth Temperature"},
                  category_orders={"intensity": ["Low", "Medium", "High"]})  # Assuming 'Low', 'Medium', 'High' are the intensity levels
fig.update_layout(paper_bgcolor="#000000",
                  plot_bgcolor="#000000",
                  height=1000, 
                  width=2000)
fig.show()

In [144]:
# Solar Flare Impact Visualization:
fig = px.line(merged_df, x="date", y=["temperature_2m_smooth", "solar_flare"],
              title="Earth Temperature and Solar Flare Events",
              labels={"date": "Date", "temperature_2m": "Earth Temperature", "solar_flare": "Solar Flare"})
fig.update_layout(paper_bgcolor="#000000",
                  plot_bgcolor="#000000",
                  height=1000, 
                  width=2000)
fig.show()

In [148]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5450 entries, 2079 to 2078
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      5450 non-null   object 
 1   min_temp                  5449 non-null   float64
 2   max_temp                  5449 non-null   float64
 3   pressure                  5450 non-null   float64
 4   temperature_2m            5450 non-null   float64
 5   relative_humidity_2m      5450 non-null   float64
 6   rain                      5450 non-null   float64
 7   direct_radiation_instant  5450 non-null   float64
 8   classtype                 5450 non-null   object 
 9   intensity                 5450 non-null   object 
 10  solar_flare               5450 non-null   int32  
 11  temperature_2m_smooth     5444 non-null   float64
dtypes: float64(8), int32(1), object(3)
memory usage: 532.2+ KB


In [155]:
# ARIMA Model:
# Choose order (p, d, q) based on ACF/PACF plots or grid search
model = ARIMA(merged_df["temperature_2m"], order=(5, 1, 0))
model_fit = model.fit()
print(model_fit.summary())

c:\Users\ansam\miniconda3\envs\healthcare_ds\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

c:\Users\ansam\miniconda3\envs\healthcare_ds\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

c:\Users\ansam\miniconda3\envs\healthcare_ds\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.



                               SARIMAX Results                                
Dep. Variable:         temperature_2m   No. Observations:                 5450
Model:                 ARIMA(5, 1, 0)   Log Likelihood                9280.764
Date:                Mon, 20 May 2024   AIC                         -18549.527
Time:                        15:56:49   BIC                         -18509.908
Sample:                             0   HQIC                        -18535.702
                               - 5450                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0368      0.011     -3.480      0.001      -0.058      -0.016
ar.L2         -0.1119      0.009    -12.278      0.000      -0.130      -0.094
ar.L3         -0.0578      0.012     -4.818      0.0

In [153]:
# Forecasting:
predictions = model_fit.predict(start=len(merged_df), end=len(merged_df)+30)
fig = px.line(merged_df, x="date", y="temperature_2m_smooth", title="Earth Temperature Forecast")
fig.add_scatter(x=merged_df["date"][-30:], y=predictions, name="Forecast")
fig.show()

c:\Users\ansam\miniconda3\envs\healthcare_ds\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\ansam\miniconda3\envs\healthcare_ds\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



In [ ]:
# Analysis:

# 1. Impact of Solar Flares on Earth Temperature:
#   - Calculate average Earth temperature before, during, and after solar flares.
#   - Perform statistical tests (e.g., t-test) to compare temperature differences.

# 2. Temporal Correlation:
#   - Calculate the correlation coefficient between Earth temperature and the occurrence of solar flares over time.
#   - Determine if there is a statistically significant correlation.

# 3. Solar Flare Intensity:
#   - Group solar flares by intensity and analyze the impact of different intensity levels on Earth temperature.
#   - Visualize the relationship between flare intensity and temperature changes.

# 4. Additional Analysis:
#   - Explore the relationship between solar flares and other weather parameters (e.g., wind speed, pressure).
#   - Investigate the impact of solar flares on specific regions of Earth.


In [154]:
flare_temp = merged_df[merged_df["solar_flare"] == 1]["temperature_2m"].mean()
before_flare_temp = merged_df[merged_df["solar_flare"] == 0]["temperature_2m"].mean()
after_flare_temp = merged_df[merged_df["solar_flare"] == 1]["temperature_2m"].shift(-1).fillna(method='bfill').mean()

# Perform t-test to compare temperature differences:
t_stat, p_value = stats.ttest_ind(merged_df[merged_df["solar_flare"] == 1]["temperature_2m"], 
                                 merged_df[merged_df["solar_flare"] == 0]["temperature_2m"])

print(f"Average Earth temperature during flares: {flare_temp:.2f}")
print(f"Average Earth temperature before flares: {before_flare_temp:.2f}")
print(f"Average Earth temperature after flares: {after_flare_temp:.2f}")
print(f"T-statistic: {t_stat:.2f}, P-value: {p_value:.3f}")

# 2. Temporal Correlation:
#   - Calculate the correlation coefficient between Earth temperature and the occurrence of solar flares over time.
#   - Determine if there is a statistically significant correlation.
correlation = merged_df["temperature_2m"].corr(merged_df["solar_flare"])
print(f"Correlation between Earth temperature and solar flares: {correlation:.2f}")

# Interpretation:
# - Do solar flares have a statistically significant impact on Earth temperature?
# - Are there specific types of flares or intensities that have a stronger impact?
# - How does the timing of solar flares relate to temperature changes?

Average Earth temperature during flares: 0.60
Average Earth temperature before flares: 0.60
Average Earth temperature after flares: 0.60
T-statistic: -0.66, P-value: 0.508
Correlation between Earth temperature and solar flares: -0.01
